# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [5]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    words = set(text)
    vocab_to_int = {word:idx for idx, word in enumerate(sorted(words))}
    int_to_vocab = {idx:word for idx, word in enumerate(sorted(words))}
    return vocab_to_int, int_to_vocab


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [7]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    return {
        ".": "||Period||",
        ",": "||Comma||",
        "\"": "||Quotation_mark||",
        ";": "||Semicolon||",
        "!": "||Exclamation_mark||",
        "?": "||Question_mark||",
        "(": "||Left_Parentheses||",
        ")": "||Right_Parentheses||",
        "--": "||Dash||",
        "\n": "||Return||"
    }

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [10]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

In [16]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    input_word_indices = tf.placeholder(name="input", shape=(None, None), dtype=tf.int32)
    target_word_indices = tf.placeholder(name="target", shape=(None, None), dtype=tf.int32)
    learning_rate = tf.placeholder(name="learning_rate", dtype=tf.float32)
    return input_word_indices, target_word_indices, learning_rate


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [21]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    layers = 1
    single_cell = tf.contrib.rnn.LSTMCell(rnn_size)
    multi_cell = tf.contrib.rnn.MultiRNNCell([single_cell] * layers)
    initial_state = tf.identity(multi_cell.zero_state(batch_size=batch_size, dtype=tf.float32), name="initial_state")
    return multi_cell, initial_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [22]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    lookup_table = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1, 1))
    return tf.nn.embedding_lookup(lookup_table, input_data)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [23]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, final_state = tf.nn.dynamic_rnn(cell, inputs, dtype=tf.float32)
    return outputs, tf.identity(final_state, name="final_state")


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [280]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    # TODO: Implement Function
    embedding = get_embed(input_data, vocab_size, embed_dim)
    rnn_output, final_state = build_rnn(cell, embedding)
    dense_output = tf.layers.dense(rnn_output, vocab_size)
    return dense_output, final_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 3, 2)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2], [ 7  8], [13 14]]
    # Batch of targets
    [[ 2  3], [ 8  9], [14 15]]
  ]

  # Second Batch
  [
    # Batch of Input
    [[ 3  4], [ 9 10], [15 16]]
    # Batch of targets
    [[ 4  5], [10 11], [16 17]]
  ]

  # Third Batch
  [
    # Batch of Input
    [[ 5  6], [11 12], [17 18]]
    # Batch of targets
    [[ 6  7], [12 13], [18  1]]
  ]
]
```

Notice that the last target value in the last batch is the first input value of the first batch. In this case, `1`. This is a common technique used when creating sequence batches, although it is rather unintuitive.

In [270]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function
    # So. batch_size refers to what?? I don't know why they
    # made it so that two things in the example are _both_ 3 (the output data above
    # is 3x2x3x2) Ugh. OK, at least I get the first 2 and the second 2.
    # is 3x(i/t)x3xsequence (i/t) is basically input vs target
    # I suppose the easiest way to replicate the behaviour I see above is this:
    # We get passed in 5000, 128, 5 and are expected to produce 7x2x128x5. Fine.
    num_batches = len(int_text) // (batch_size * seq_length)
    
    realsz = num_batches * (batch_size * seq_length)
    input_data = int_text[0:realsz]
    target_data = input_data[1:] + int_text[0:1]
    
    def sequence(data):
        return [data[x:x+seq_length] for x in range(0, realsz, seq_length)]

    input_sequences = sequence(input_data)
    target_sequences = sequence(target_data)

    # So, we need batches of these sequences, but sort of transposed. So,
    # instead, let's make it by num_batches, then we'll zip.
    def batch(data):
        return [_ for _ in zip(*[data[x:x+num_batches] for x in range(0, len(data), num_batches)])]

    input_batch = batch(input_sequences)
    target_batch = batch(target_sequences)

    stitched = [_ for _ in zip(input_batch, target_batch)]
    
    return np.array(stitched)
"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [284]:
# Number of Epochs
num_epochs = 100
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 16
# Embedding Dimension Size
embed_dim = 256
# Sequence Length
seq_length = 8
# Learning Rate
learning_rate = 0.01
# Show stats for every n number of batches
show_every_n_batches = 1

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [285]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forums](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [286]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/269   train_loss = 8.823
Epoch   0 Batch    1/269   train_loss = 8.797
Epoch   0 Batch    2/269   train_loss = 8.768
Epoch   0 Batch    3/269   train_loss = 8.714
Epoch   0 Batch    4/269   train_loss = 8.679
Epoch   0 Batch    5/269   train_loss = 8.605
Epoch   0 Batch    6/269   train_loss = 8.500
Epoch   0 Batch    7/269   train_loss = 8.391
Epoch   0 Batch    8/269   train_loss = 8.294
Epoch   0 Batch    9/269   train_loss = 8.100
Epoch   0 Batch   10/269   train_loss = 8.032
Epoch   0 Batch   11/269   train_loss = 7.868
Epoch   0 Batch   12/269   train_loss = 7.699
Epoch   0 Batch   13/269   train_loss = 7.583
Epoch   0 Batch   14/269   train_loss = 7.284
Epoch   0 Batch   15/269   train_loss = 7.243
Epoch   0 Batch   16/269   train_loss = 7.126
Epoch   0 Batch   17/269   train_loss = 6.993
Epoch   0 Batch   18/269   train_loss = 6.797
Epoch   0 Batch   19/269   train_loss = 6.706
Epoch   0 Batch   20/269   train_loss = 6.711
Epoch   0 Batch   21/269   train_l

Epoch   0 Batch  189/269   train_loss = 5.615
Epoch   0 Batch  190/269   train_loss = 5.754
Epoch   0 Batch  191/269   train_loss = 5.561
Epoch   0 Batch  192/269   train_loss = 5.496
Epoch   0 Batch  193/269   train_loss = 5.454
Epoch   0 Batch  194/269   train_loss = 5.804
Epoch   0 Batch  195/269   train_loss = 5.578
Epoch   0 Batch  196/269   train_loss = 5.890
Epoch   0 Batch  197/269   train_loss = 5.913
Epoch   0 Batch  198/269   train_loss = 6.017
Epoch   0 Batch  199/269   train_loss = 6.267
Epoch   0 Batch  200/269   train_loss = 6.240
Epoch   0 Batch  201/269   train_loss = 5.445
Epoch   0 Batch  202/269   train_loss = 5.470
Epoch   0 Batch  203/269   train_loss = 5.654
Epoch   0 Batch  204/269   train_loss = 5.565
Epoch   0 Batch  205/269   train_loss = 5.871
Epoch   0 Batch  206/269   train_loss = 5.678
Epoch   0 Batch  207/269   train_loss = 5.540
Epoch   0 Batch  208/269   train_loss = 5.667
Epoch   0 Batch  209/269   train_loss = 5.678
Epoch   0 Batch  210/269   train_l

Epoch   1 Batch  109/269   train_loss = 5.191
Epoch   1 Batch  110/269   train_loss = 5.268
Epoch   1 Batch  111/269   train_loss = 5.285
Epoch   1 Batch  112/269   train_loss = 5.381
Epoch   1 Batch  113/269   train_loss = 4.924
Epoch   1 Batch  114/269   train_loss = 5.074
Epoch   1 Batch  115/269   train_loss = 5.143
Epoch   1 Batch  116/269   train_loss = 5.002
Epoch   1 Batch  117/269   train_loss = 4.953
Epoch   1 Batch  118/269   train_loss = 5.246
Epoch   1 Batch  119/269   train_loss = 5.143
Epoch   1 Batch  120/269   train_loss = 5.215
Epoch   1 Batch  121/269   train_loss = 5.129
Epoch   1 Batch  122/269   train_loss = 5.183
Epoch   1 Batch  123/269   train_loss = 5.007
Epoch   1 Batch  124/269   train_loss = 5.229
Epoch   1 Batch  125/269   train_loss = 4.857
Epoch   1 Batch  126/269   train_loss = 4.663
Epoch   1 Batch  127/269   train_loss = 5.068
Epoch   1 Batch  128/269   train_loss = 5.090
Epoch   1 Batch  129/269   train_loss = 5.253
Epoch   1 Batch  130/269   train_l

Epoch   2 Batch   30/269   train_loss = 4.944
Epoch   2 Batch   31/269   train_loss = 4.840
Epoch   2 Batch   32/269   train_loss = 4.629
Epoch   2 Batch   33/269   train_loss = 4.689
Epoch   2 Batch   34/269   train_loss = 4.945
Epoch   2 Batch   35/269   train_loss = 4.931
Epoch   2 Batch   36/269   train_loss = 4.755
Epoch   2 Batch   37/269   train_loss = 4.795
Epoch   2 Batch   38/269   train_loss = 4.852
Epoch   2 Batch   39/269   train_loss = 4.986
Epoch   2 Batch   40/269   train_loss = 4.672
Epoch   2 Batch   41/269   train_loss = 4.744
Epoch   2 Batch   42/269   train_loss = 5.423
Epoch   2 Batch   43/269   train_loss = 5.084
Epoch   2 Batch   44/269   train_loss = 4.846
Epoch   2 Batch   45/269   train_loss = 4.693
Epoch   2 Batch   46/269   train_loss = 5.229
Epoch   2 Batch   47/269   train_loss = 4.626
Epoch   2 Batch   48/269   train_loss = 5.203
Epoch   2 Batch   49/269   train_loss = 4.889
Epoch   2 Batch   50/269   train_loss = 5.200
Epoch   2 Batch   51/269   train_l

Epoch   2 Batch  213/269   train_loss = 4.837
Epoch   2 Batch  214/269   train_loss = 4.954
Epoch   2 Batch  215/269   train_loss = 5.063
Epoch   2 Batch  216/269   train_loss = 5.047
Epoch   2 Batch  217/269   train_loss = 4.837
Epoch   2 Batch  218/269   train_loss = 4.737
Epoch   2 Batch  219/269   train_loss = 4.484
Epoch   2 Batch  220/269   train_loss = 4.734
Epoch   2 Batch  221/269   train_loss = 4.735
Epoch   2 Batch  222/269   train_loss = 4.776
Epoch   2 Batch  223/269   train_loss = 4.883
Epoch   2 Batch  224/269   train_loss = 4.784
Epoch   2 Batch  225/269   train_loss = 5.188
Epoch   2 Batch  226/269   train_loss = 5.053
Epoch   2 Batch  227/269   train_loss = 4.600
Epoch   2 Batch  228/269   train_loss = 4.731
Epoch   2 Batch  229/269   train_loss = 4.882
Epoch   2 Batch  230/269   train_loss = 5.073
Epoch   2 Batch  231/269   train_loss = 4.842
Epoch   2 Batch  232/269   train_loss = 4.637
Epoch   2 Batch  233/269   train_loss = 4.792
Epoch   2 Batch  234/269   train_l

...

Epoch  96 Batch  210/269   train_loss = 3.598
Epoch  96 Batch  211/269   train_loss = 3.698
Epoch  96 Batch  212/269   train_loss = 4.093
Epoch  96 Batch  213/269   train_loss = 3.641
Epoch  96 Batch  214/269   train_loss = 3.668
Epoch  96 Batch  215/269   train_loss = 3.905
Epoch  96 Batch  216/269   train_loss = 3.950
Epoch  96 Batch  217/269   train_loss = 3.602
Epoch  96 Batch  218/269   train_loss = 3.760
Epoch  96 Batch  219/269   train_loss = 3.576
Epoch  96 Batch  220/269   train_loss = 3.959
Epoch  96 Batch  221/269   train_loss = 3.587
Epoch  96 Batch  222/269   train_loss = 3.765
Epoch  96 Batch  223/269   train_loss = 3.576
Epoch  96 Batch  224/269   train_loss = 3.853
Epoch  96 Batch  225/269   train_loss = 3.894
Epoch  96 Batch  226/269   train_loss = 3.687
Epoch  96 Batch  227/269   train_loss = 3.450
Epoch  96 Batch  228/269   train_loss = 3.791
Epoch  96 Batch  229/269   train_loss = 3.964
Epoch  96 Batch  230/269   train_loss = 3.788
Epoch  96 Batch  231/269   train_l

Epoch  97 Batch  128/269   train_loss = 3.663
Epoch  97 Batch  129/269   train_loss = 3.653
Epoch  97 Batch  130/269   train_loss = 3.805
Epoch  97 Batch  131/269   train_loss = 3.410
Epoch  97 Batch  132/269   train_loss = 3.808
Epoch  97 Batch  133/269   train_loss = 3.448
Epoch  97 Batch  134/269   train_loss = 3.889
Epoch  97 Batch  135/269   train_loss = 3.595
Epoch  97 Batch  136/269   train_loss = 3.416
Epoch  97 Batch  137/269   train_loss = 3.778
Epoch  97 Batch  138/269   train_loss = 3.556
Epoch  97 Batch  139/269   train_loss = 3.767
Epoch  97 Batch  140/269   train_loss = 3.838
Epoch  97 Batch  141/269   train_loss = 4.138
Epoch  97 Batch  142/269   train_loss = 3.522
Epoch  97 Batch  143/269   train_loss = 3.706
Epoch  97 Batch  144/269   train_loss = 3.902
Epoch  97 Batch  145/269   train_loss = 3.704
Epoch  97 Batch  146/269   train_loss = 3.925
Epoch  97 Batch  147/269   train_loss = 3.505
Epoch  97 Batch  148/269   train_loss = 3.930
Epoch  97 Batch  149/269   train_l

Epoch  98 Batch   46/269   train_loss = 3.753
Epoch  98 Batch   47/269   train_loss = 3.363
Epoch  98 Batch   48/269   train_loss = 3.943
Epoch  98 Batch   49/269   train_loss = 3.887
Epoch  98 Batch   50/269   train_loss = 3.870
Epoch  98 Batch   51/269   train_loss = 3.791
Epoch  98 Batch   52/269   train_loss = 3.733
Epoch  98 Batch   53/269   train_loss = 3.996
Epoch  98 Batch   54/269   train_loss = 3.681
Epoch  98 Batch   55/269   train_loss = 3.730
Epoch  98 Batch   56/269   train_loss = 3.518
Epoch  98 Batch   57/269   train_loss = 3.969
Epoch  98 Batch   58/269   train_loss = 3.584
Epoch  98 Batch   59/269   train_loss = 3.662
Epoch  98 Batch   60/269   train_loss = 3.843
Epoch  98 Batch   61/269   train_loss = 3.924
Epoch  98 Batch   62/269   train_loss = 3.609
Epoch  98 Batch   63/269   train_loss = 3.900
Epoch  98 Batch   64/269   train_loss = 3.620
Epoch  98 Batch   65/269   train_loss = 3.770
Epoch  98 Batch   66/269   train_loss = 3.635
Epoch  98 Batch   67/269   train_l

Epoch  98 Batch  225/269   train_loss = 3.876
Epoch  98 Batch  226/269   train_loss = 3.700
Epoch  98 Batch  227/269   train_loss = 3.444
Epoch  98 Batch  228/269   train_loss = 3.786
Epoch  98 Batch  229/269   train_loss = 3.959
Epoch  98 Batch  230/269   train_loss = 3.830
Epoch  98 Batch  231/269   train_loss = 3.698
Epoch  98 Batch  232/269   train_loss = 3.726
Epoch  98 Batch  233/269   train_loss = 3.852
Epoch  98 Batch  234/269   train_loss = 3.745
Epoch  98 Batch  235/269   train_loss = 4.060
Epoch  98 Batch  236/269   train_loss = 3.628
Epoch  98 Batch  237/269   train_loss = 3.276
Epoch  98 Batch  238/269   train_loss = 3.607
Epoch  98 Batch  239/269   train_loss = 4.020
Epoch  98 Batch  240/269   train_loss = 3.571
Epoch  98 Batch  241/269   train_loss = 3.936
Epoch  98 Batch  242/269   train_loss = 3.612
Epoch  98 Batch  243/269   train_loss = 3.903
Epoch  98 Batch  244/269   train_loss = 3.685
Epoch  98 Batch  245/269   train_loss = 3.743
Epoch  98 Batch  246/269   train_l

Epoch  99 Batch  137/269   train_loss = 3.817
Epoch  99 Batch  138/269   train_loss = 3.595
Epoch  99 Batch  139/269   train_loss = 3.739
Epoch  99 Batch  140/269   train_loss = 3.853
Epoch  99 Batch  141/269   train_loss = 4.118
Epoch  99 Batch  142/269   train_loss = 3.541
Epoch  99 Batch  143/269   train_loss = 3.676
Epoch  99 Batch  144/269   train_loss = 3.896
Epoch  99 Batch  145/269   train_loss = 3.695
Epoch  99 Batch  146/269   train_loss = 3.931
Epoch  99 Batch  147/269   train_loss = 3.483
Epoch  99 Batch  148/269   train_loss = 3.873
Epoch  99 Batch  149/269   train_loss = 3.636
Epoch  99 Batch  150/269   train_loss = 3.902
Epoch  99 Batch  151/269   train_loss = 3.777
Epoch  99 Batch  152/269   train_loss = 3.739
Epoch  99 Batch  153/269   train_loss = 3.818
Epoch  99 Batch  154/269   train_loss = 3.703
Epoch  99 Batch  155/269   train_loss = 3.601
Epoch  99 Batch  156/269   train_loss = 4.122
Epoch  99 Batch  157/269   train_loss = 3.870
Epoch  99 Batch  158/269   train_l

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [290]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [291]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [292]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    return [loaded_graph.get_tensor_by_name(name + ":0") for name in ["input", "initial_state", "final_state", "probs"]]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [293]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    # Ugh. Different values for test vs the generate below.
    p = probabilities
    if probabilities.ndim > 1:
        p = np.reshape(probabilities, probabilities.size)
    idx = (np.random.choice(np.arange(p.size), 1, True, p))[0]
    return int_to_vocab[idx]
    


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [294]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        pred_word = pick_word(probabilities[(0, dyn_seq_length-1)], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak:--
homer_simpson:(singing) ayyy! hey(happy noise chuckle) i'll never follow me with it a beer that isotopes!
moe_szyslak:(back in me in, my wallet!
joey_kramer: miracle tree!
carl_carlson: before my cruiser.
homer_simpson: no, sweetheart. i had i'll teach you?
carl_carlson: oh thanks, that's it, if i get your wife out with nothin'. while!
homer_simpson: who'll gee, let me you all two there might be gentle!
don't help, always here sure over cars guy in the bow for keeping out, the clientele dress up out back, i had handed 'em for you get him, sisters i never throw wanted, great with here?
carl_carlson: come about me!
homer_simpson:(victorious chuckle)
moe_szyslak: whoa!
barney_gumble: i'm selling, the matter, but who be bedridden for an english is a lot time.. i've gotta teach ya!


homer_simpson:(leans from tv his favorite) hot blooded / i ain't real register easy, homer here


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.